In [73]:
import numpy as np
from sympy import *
from scipy.linalg import eigh
    
# Tropical addition
# num - num
# n-vector - n-vector
# nm-matrix - nm-matrix
def tplus(a, b):
    if (mode == 0):
        if (len(np.shape(a)) == 2 and np.shape(a) == np.shape(b)):
            n, m = np.shape(a)
            out = np.zeros(shape=(n,m))
            
            for i in range(n):
                for j in range(m):
                    out[i][j] = min(a[i][j], b[i][j])
            return out
        
        elif (len(np.shape(a)) == len(np.shape(b)) == 1):
            n = np.shape(a)[0]
            out = np.zeros(n)
            
            for i in range(n):
                out[i] = min(a[i], b[i])
            return out
        return min(a, b)
    else:
        if (len(np.shape(a)) == 2 and np.shape(a) == np.shape(b)):
            n, m = np.shape(a)
            out = np.zeros(shape=(n,m))
            
            for i in range(n):
                for j in range(m):
                    out[i][j] = max(a[i][j], b[i][j])
            return out
        
        elif (len(np.shape(a)) == len(np.shape(b)) == 1):
            n = np.shape(a)[0]
            out = np.zeros(n)
            for i in range(n):
                out[i] = max(a[i], b[i])
            return out
        return max(a, b)

# Tropical multiplication
# num - num
# scalar - vector
# scalar - matrix
# nm-matrix - mx-matrix
# nm-matrix - m-vector
def ttimes(a, b):
    if (len(np.shape(a)) == 2 and np.shape(a)[1] == np.shape(b)[0]):
        n, m = np.shape(a)
        if(len(np.shape(b))==1):
            b = b.reshape(len(b),1)
        m, x = np.shape(b)
        out = np.zeros(shape=(n,x))
        
        for i in range(n):
            for j in range(x):
                val = ttimes(a[i][0],b[0][j])
                
                for k in range(m):
                    val = tplus(val, ttimes(a[i][k],b[k][j]))
                out[i][j] = val
        return out
    
    elif (len(np.shape(b)) == 1):
        m = np.shape(b)[0]
        out = np.zeros(m)
        
        for i in range(m):
            out[i] = ttimes(b[i],a)
        return out
    
    elif (len(np.shape(b)) == 2):
        n, m = np.shape(b)
        out = np.zeros(shape=(n, m))
        
        for i in range(n):
            for j in range(m):
                out[i][j] = ttimes(b[i][j], a)
        return out
    
    return (a + b) if (mode2==0) else (a * b)

# Tropical dot product
# n-vector - n-vector
def tdot(a, b):
    val = ttimes(a[0], b[0])
    for i in range(len(a)):
        if mode == 0:
            val = min(val, ttimes(a[i],b[i]))
        else:
            val = max(val, ttimes(a[i],b[i]))
    return val

# Tropical exponentiation
def texp(a, b):
    return (a * b) if (mode2==0) else (a**b)

# Calculate tropical eigenvalue of a square matrix
def teigval(matrix):
    n, m = np.shape(matrix)
    assert n == m
    ma = matrix
    cur = matrix
    for i in range(1,n):
        ma = ttimes(ma, matrix)
        cur = tplus(cur, (1/(i+1))*ma)
    val = cur[0][0]
    for i in range(n):
        val = tplus(val, cur[i][i])
    return val

# Calculate tropical eigenvectors of a square matrix
def teigvec(matrix):
    n, m = np.shape(matrix)
    assert n == m
    
    eigval = teigval(matrix)
    matrix = ttimes(texp(eigval,-1), matrix)
    
    ma = matrix
    cur = matrix
    
    for i in range(1,n):
        ma = ttimes(ma, matrix)
        cur = tplus(cur, ma)
    out = []
    cur = cur.transpose()
    
    for i in range(n):
        if (cur[i][i] == 0):
            out.append(cur[i])
    return basis(out)

# Calculate basis of given list of vectors
def basis(vectors):
    count = 1
    n = len(vectors)
    out = []
    if (n==0):
        return out
    
    out.append(ttimes((texp(vectors[0][0],-1)),vectors[0]))
    
    for i in range(1,n):
        vectors[i] = ttimes(texp(vectors[i][0],-1),vectors[i])
        matches = 0
        
        for j in range(0,len(out)):
            if ((out[j] == vectors[i]).all()):
                matches = 1
        if (matches == 0):
            out.append(vectors[i])
    return out

# Compute transfer matrix
def transfer_matrix(A, h):
    n, m = np.shape(A)
    assert n == m
    out = np.zeros(shape=(n,m))

    for i in range(n):
        for j in range(m):
            out[i][j] = np.exp(A[i][j]*h)
    return out

# Compute transfer PF eval/evecs
def transfer_ev(A, h):
    n,m = np.shape(A)
    E = transfer_matrix(A, h)
    val,vec = eigh(E)
    print()
    print(h)
    print(E)
    print(val)
    print(vec)
    
    vec = vec.T
    ind = n-1
    if (-1*val[0] > val[ind]):
        ind = 0
    
    ove = vec[ind]
    if (ove[0] < 0):
        ove = -1*ove
    for i in range(len(ove)):
        ove[i] = (1/h)*np.log(ove[i])
    ova = val[ind]
    ova = (1/h)*np.log(ova)
    return ova, ove

# Compute transfer asymptote
def matrix_asymptote(A):
    h = Symbol('h')
    
    n, m = np.shape(A)
    assert n == m
    out = np.zeros(shape=(n, m))
    
    for i in range(n):
        for j in range(m):
            out[i][j] = limit(exp((1/h)*A[i][j]),h,0)
    
    return out
            

In [55]:
# 0 : min
# 1 : max
mode = 1

# 0 : plus
# 1 : times
mode2 = 0

In [56]:
a = np.zeros(3)
b = np.zeros(3)
c = np.zeros(3)
a[0], a[1], a[2] = 1, 1, 1/5
b[0], b[1], b[2] = 1, 3, 1/2
c[0], c[1], c[2] = 5, 2, 1

In [69]:
m1 = np.zeros(shape=(3,3))
m2 = np.zeros(shape=(3,3))
m1[0], m1[1], m1[2] = a, b, c
m2[0], m2[1], m2[2] = a, b, c
m2 = m2.transpose()
m1 = np.log(m1)
print(m1)

v1 = np.zeros(shape=(3,1))
v2 = np.zeros(shape=(3,1))
v3 = np.zeros(shape=(3,1))
v1[0], v1[1], v1[2] = 0, -1, -1
v2[0], v2[1], v2[2] = 0, 2, 1
v3[0], v3[1], v3[2] = 0, 0, 1

[[ 0.          0.         -1.60943791]
 [ 0.          1.09861229 -0.69314718]
 [ 1.60943791  0.69314718  0.        ]]


In [70]:
print(teigval(m1))

1.0986122886681098


In [71]:
print(teigvec(m1))

[[-1.09861229 -1.09861229 -2.7080502 ]
 [-1.09861229  0.         -1.79175947]
 [ 0.51082562 -0.40546511 -1.09861229]]
[[-1.09861229 -1.09861229 -2.7080502 ]
 [-1.09861229  0.         -1.79175947]
 [ 0.51082562 -0.40546511 -1.09861229]]
[array([0.        , 1.09861229, 0.69314718])]


In [66]:
print(ttimes(m1, teigvec(m1)))

[[-2.  -2.  -2.8]
 [-2.   0.  -2.5]
 [ 2.  -1.  -2. ]]
[[-0.8 -2.  -2.8]
 [-0.5  0.  -2.5]
 [ 2.   0.  -0.8]]


ValueError: setting an array element with a sequence.

In [74]:
print(m1)
print(eigh(m1))
print()
print(transfer_ev(m1,1))
print(transfer_ev(m1,10))
print(transfer_ev(m1,100))
print(transfer_ev(m1,1000))
print(transfer_ev(m1,10000))

[[ 0.          0.         -1.60943791]
 [ 0.          1.09861229 -0.69314718]
 [ 1.60943791  0.69314718  0.        ]]
(array([-1.69763937,  0.87042105,  1.92583061]), array([[ 0.6771341 ,  0.50054819, -0.5393894 ],
       [ 0.17704966, -0.82229478, -0.54081856],
       [-0.71424284,  0.27070797, -0.64542572]]))


1
[[1.  1.  0.2]
 [1.  3.  0.5]
 [5.  2.  1. ]]
[-4.07542846  1.96777436  7.10765411]
[[ 0.68715499  0.40117128  0.60570589]
 [ 0.10604374 -0.88017402  0.46265367]
 [-0.71872995  0.25368346  0.64735768]]
(1.961172246778648, array([-0.50136075, -0.77077652, -0.43485631]))

10
[[1.000000e+00 1.000000e+00 1.024000e-07]
 [1.000000e+00 5.904900e+04 9.765625e-04]
 [9.765625e+06 1.024000e+03 1.000000e+00]]
[-9765624.05326025    59048.99914102  9765626.05411923]
[[ 7.07106777e-01  1.04862052e-04  7.07106777e-01]
 [ 7.36279196e-05 -9.99999995e-01  7.46694171e-05]
 [-7.07106781e-01  7.36449984e-07  7.07106781e-01]]
(1.609437923228281, array([-0.03465736, -0.950244  , -0.03465736]))

100

/home/balazs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:191: RuntimeWarning: divide by zero encountered in log


ValueError: array must not contain infs or NaNs